# QSM Reconstruction

In [ ]:
!run_2_qsm.py bids-new qsm-new --premade body --auto_yes --slurm a_barth general

In [16]:
!for f in `find qsm-new3/qsm_final -name '*.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; cp "${f}" "bids-new2/${sub}/${ses}/extra_data/"; done

In [15]:
!for f in `find qsm-new3/ -wholename '*numpy_nibabel_twopass/mapflow/*/*echo-02*twopass.nii*'`; do sub="${f/*sub/sub}"; sub="${sub/_ses*/}"; ses="${f/*_ses/ses}"; ses="${ses/_*/}"; f2="${f/twopass1/twopass3}"; f2="${f2/echo-02/echo-04}"; ~/qsmxt/interfaces/nipype_interface_nonzeroaverage.py "${f}" "${f2}" "bids-new2/${sub}/${ses}/extra_data/${sub}_${ses}_qsm_echo2-and-echo4.nii"; done

/home/uqaste15/qsmxt/interfaces/nipype_interface_nonzeroaverage.py:33: RuntimeWarning: divide by zero encountered in true_divide
  final = data.sum(0) / mask.sum(0)
/home/uqaste15/qsmxt/interfaces/nipype_interface_nonzeroaverage.py:33: RuntimeWarning: invalid value encountered in true_divide
  final = data.sum(0) / mask.sum(0)


# Imports

In [1]:
import os
import glob
import nibabel as nib
import nilearn.image
import numpy as np
import enum
import json
import cv2
import scipy
from scipy.ndimage import binary_dilation, generate_binary_structure, label

# Locate data

In [2]:
bids_dir = "bids-new"

In [3]:
session_dirs = []
for json_path in sorted(glob.glob(os.path.join(bids_dir, "sub*", "ses*", "anat", "*echo-01*mag*json"))):
    with open(json_path, 'r') as json_file:
        json_data = json.load(json_file)
        if json_data['ProtocolName'] == "t2starME_qsm_tra_Iso1.4mm_INPHASE_bipolar_RUN_THIS_ONE":
            session_dirs.append(os.sep.join(os.path.split(json_path)[0].split(os.sep)[:-1]))
#session_dirs = [session_dir for session_dir in session_dirs if any(x in session_dir for x in ['sub-z1571613', 'sub-z2900812', 'sub-z2952003'])]
print(f"{len(session_dirs)} sessions found")

24 sessions found


In [4]:
extra_files = sum((glob.glob(os.path.join(session_dir, "extra_data", "*.nii*")) for session_dir in session_dirs), [])

qsm_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*qsm_echo2-and-echo4.*")) for session_dir in session_dirs), []))
t2s_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*t2starmap.nii*")) for session_dir in session_dirs), []))
r2s_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "*r2starmap.nii*")) for session_dir in session_dirs), []))
mag_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "magnitude_combined.nii")) for session_dir in session_dirs), []))
swi_files = sorted(sum((glob.glob(os.path.join(session_dir, "extra_data", "swi.nii")) for session_dir in session_dirs), []))

gre_seg_raw_files = [extra_file for extra_file in extra_files if all(pattern in extra_file for pattern in ['_segmentation.', 'run'])]
gre_seg_clean_files = [seg_file.replace(".nii", "_clean.nii") for seg_file in gre_seg_raw_files if os.path.exists(seg_file.replace(".nii", "_clean.nii"))]

t1_files = [extra_file for extra_file in extra_files if any(pattern in extra_file for pattern in ['_T1w', '_t1_tra']) and not any(pattern in extra_file for pattern in ['_Pelvis_', '.json', '_resampled'])]
t1_files = [t1_file.replace("_resampled", "") for t1_file in t1_files]
t1_resampled_files = [t1_file.replace(".nii", "_resampled.nii") for t1_file in t1_files if os.path.exists(t1_file.replace(".nii", "_resampled.nii"))]

ct_files = [extra_file for extra_file in extra_files if 'resliced' in extra_file and any(pattern in extra_file for pattern in ['_na_', '_Pelvis_', '_NA']) and not any(pattern in extra_file for pattern in ['_t1_tra_', 'ATX', 'AXT', 'ROI', 'segmentation', '.json'])]
ct_seg_raw_files = sum((glob.glob(ct_file.replace(".nii", "_segmentation.nii")) for ct_file in ct_files), [])
ct_seg_clean_files = [ct_file.replace("_segmentation", "_segmentation_clean") for ct_file in ct_seg_raw_files if os.path.exists(ct_file)]

print(f"{len(ct_files)} CT images found.")
print(f"{len(ct_seg_raw_files)} raw CT segmentations found.")
print(f"{len(ct_seg_clean_files)} clean CT segmentations found.")
print(f"{len(qsm_files)} QSM images found.")
print(f"{len(mag_files)} magnitude images found.")
print(f"{len(t2s_files)} T2* maps found.")
print(f"{len(r2s_files)} R2* maps found.")
print(f"{len(swi_files)} SWI maps found.")
print(f"{len(t1_files)} T1w files found.")
print(f"{len(t1_resampled_files)} resampled T1w files found.")
print(f"{len(gre_seg_raw_files)} raw GRE segmentations found.")
print(f"{len(gre_seg_clean_files)} clean GRE segmentations found.")

24 CT images found.
24 raw CT segmentations found.
24 clean CT segmentations found.
24 QSM images found.
24 magnitude images found.
24 T2* maps found.
24 R2* maps found.
24 SWI maps found.
24 T1w files found.
24 resampled T1w files found.
24 raw GRE segmentations found.
24 clean GRE segmentations found.


# Clean up segmentations

In [68]:
'''
# 1 becomes 3
# 2 becomes 1
# 3 becomes 2

for i in range(len(ct_seg_raw_files)):
    print(f"Loading {ct_seg_raw_files[i]}")
    seg_nii = nib.load(ct_seg_raw_files[i])
    seg = np.array(np.round(seg_nii.get_fdata()), dtype=int)

    print(f"Values before: {np.unique(seg)}")
    #seg[seg == 1] = 3
    #seg[seg == 2] = 1
    #seg[seg == 3] = 2

    seg[seg == 1] = 4 # 3
    seg[seg == 2] = 1
    seg[seg == 3] = 2
    seg[seg == 4] = 3
    
    print(f"Values after: {np.unique(seg)}")

    filename = ct_seg_raw_files[i].replace("_segmentation.nii", "_segmentation_fixed.nii")
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename)
    print(filename)
'''

Loading bids-new/sub-z1571613/ses-20230519/extra_data/5_NA_20230519084109_5_resliced_segmentation.nii
Values before: [0 1 2 3]
Values after: [0 1 2 3]
bids-new/sub-z1571613/ses-20230519/extra_data/5_NA_20230519084109_5_resliced_segmentation_fixed.nii
Loading bids-new/sub-z2900812/ses-20230523/extra_data/5_NA_20230523084412_5_resliced_segmentation.nii
Values before: [0 1 2 3]
Values after: [0 1 2 3]
bids-new/sub-z2900812/ses-20230523/extra_data/5_NA_20230523084412_5_resliced_segmentation_fixed.nii
Loading bids-new/sub-z2952003/ses-20230614/extra_data/5_NA_20230614092610_5_resliced_segmentation.nii
Values before: [0 1 2 3]
Values after: [0 1 2 3]
bids-new/sub-z2952003/ses-20230614/extra_data/5_NA_20230614092610_5_resliced_segmentation_fixed.nii


In [6]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    qsm_nii = nib.load(qsm_files[i])
    seg_nii = nib.load(gre_seg_raw_files[i])

    # get image data
    qsm = qsm_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = qsm[seg == SegType.PROSTATE.value]

    # remove gold seed values close to the mean and smooth
    seed_mask = np.logical_and(seg == SegType.GOLD_SEED.value, np.logical_or(qsm > prostate_values.mean() + 1*prostate_values.std(), qsm < prostate_values.mean() - 1*prostate_values.std()))
    smoothed = scipy.ndimage.gaussian_filter(seed_mask * 1.0, sigma=[2,2,2])
    seed_mask[smoothed > 0.16] = 1

    # automated calcification mask
    calc_mask = np.logical_and(seg != SegType.GOLD_SEED, qsm < np.mean(prostate_values) - 3*np.std(prostate_values))
    calc_mask = np.logical_and((qsm < np.mean(prostate_values) - 2*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((qsm < np.mean(prostate_values) - 2*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((qsm < np.mean(prostate_values) - 2*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((qsm < np.mean(prostate_values) - 2*np.std(prostate_values)), binary_dilation(calc_mask))

    # remove unconnected single voxels from the mask
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0

    # combine automatic mask with manual mask
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)
    
    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{gre_seg_raw_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids-new/sub-z0034542/ses-20220715/extra_data/sub-z0034542_ses-20220715_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0182923/ses-20230705/extra_data/sub-z0182923_ses-20230705_run-01_segmentation_clean.nii
Saved bids-new/sub-z0186251/ses-20221107/extra_data/sub-z0186251_ses-20221107_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z0237546/ses-20230508/extra_data/sub-z0237546_ses-20230508_run-01_segmentation_clean.nii
Saved bids-new/sub-z0317485/ses-20230517/extra_data/sub-z0317485_ses-20230517_run-01_segmentation_clean.nii
Saved bids-new/sub-z0445614/ses-20230510/extra_data/sub-z0445614_ses-20230510_run-01_segmentation_clean.nii
Saved bids-new/sub-z0705200/ses-20230104/extra_data/sub-z0705200_ses-20230104_run-01_segmentation_clean.nii
Saved bids-new/sub-z0755228/ses-20211108/extra_data/sub-z0755228_ses-20211108_run-01_segmentation_clean.nii.gz
Saved bids-new/sub-z1167038/ses-20220315/extra_data/sub-z1167038_ses-20220315_run-01_segmentation_clean.nii.gz
Saved bids-new/s

In [71]:
class SegType(enum.Enum):
    NO_LABEL = 0
    PROSTATE = 1
    GOLD_SEED = 2
    CALCIFICATION = 3

for i in range(len(qsm_files)):
    # load files
    ct_nii = nib.load(ct_files[i])
    seg_nii = nib.load(ct_seg_raw_files[i])

    # get image data
    ct = ct_nii.get_fdata()
    seg = np.array(seg_nii.get_fdata(), dtype=np.uint8)

    # get prostate values
    prostate_values = ct[seg == SegType.PROSTATE.value]

    # remove gold seed values close to the mean and smooth
    seed_mask = np.logical_and(seg == SegType.GOLD_SEED.value, ct > prostate_values.mean() + 10*prostate_values.std())
    seed_mask = np.logical_and((ct > np.mean(prostate_values) + 8*np.std(prostate_values)), binary_dilation(seed_mask))
    seed_mask = np.logical_and((ct > np.mean(prostate_values) + 8*np.std(prostate_values)), binary_dilation(seed_mask))
    #smoothed = scipy.ndimage.gaussian_filter(seed_mask * 1.0, sigma=[2,2,2])
    #seed_mask[smoothed > 0.2] = 1

    # automated calcification mask
    calc_mask = np.logical_and(seg != SegType.GOLD_SEED, ct > np.mean(prostate_values) + 4*np.std(prostate_values))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 3*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 3*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 3*np.std(prostate_values)), binary_dilation(calc_mask))
    calc_mask = np.logical_and((ct > np.mean(prostate_values) + 3*np.std(prostate_values)), binary_dilation(calc_mask))

    # remove unconnected single voxels from the mask
    structure = generate_binary_structure(3, 1) # 3 for 3D image, 2 for full connectivity
    labeled_mask, num_features = label(calc_mask, structure)
    for idx in range(1, num_features + 1):
        if np.sum(labeled_mask == idx) == 1:
            calc_mask[labeled_mask == idx] = 0

    # combine automatic mask with manual mask
    calc_mask = np.logical_and(calc_mask, seg == SegType.CALCIFICATION.value)
    
    # setup final values
    seg[seg == SegType.GOLD_SEED.value] = 0
    seg[seg == SegType.CALCIFICATION.value] = 0
    seg[seg == SegType.PROSTATE.value] = 0

    seg[seed_mask] = 1
    seg[calc_mask] = 2
    
    # save result using original file extension
    filename = f"{ct_seg_raw_files[i].replace('.nii', '_clean.nii')}"
    nib.save(nib.Nifti1Image(seg, header=seg_nii.header, affine=seg_nii.affine), filename=filename)
    print(f"Saved {filename}")

Saved bids-new/sub-z0034542/ses-20220715/extra_data/5_na_20220715092125_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0182923/ses-20230705/extra_data/5_NA_20230705093737_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0186251/ses-20221107/extra_data/5_na_20221107081460_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0237546/ses-20230508/extra_data/7_na_20230508102530_7_resliced_segmentation_clean.nii
Saved bids-new/sub-z0317485/ses-20230517/extra_data/5_NA_20230517091908_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0445614/ses-20230510/extra_data/5_16_pelvis_16_Pelvis_20230510090050_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0705200/ses-20230104/extra_data/5_11_pelvis_11_Pelvis_20230104113938_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z0755228/ses-20211108/extra_data/Z0755228_20211109_Pelvis_2.0_HD_FoV_iMAR_20211109085844_5_resliced_segmentation_clean.nii
Saved bids-new/sub-z1167038/ses-20220315/extra_data/6_na_Pelvis_2.0_Br38_3_iMAR_20

# Combined magnitude

In [5]:
for session_dir in session_dirs:
    mag_images = sorted(glob.glob(os.path.join(session_dir, "anat", "*part-mag*nii*")))
    mag_images_even = [mag_images[i] for i in range(1, len(mag_images), 2)]
    nii = nib.load(mag_images_even[0])
    mag_4d = np.array([nib.load(mag_images_even[i]).get_fdata() for i in range(len(mag_images_even))])
    mag_combined = np.sqrt(np.sum(np.square(mag_4d), axis=0))
    nib.save(nib.Nifti1Image(mag_combined, header=nii.header, affine=nii.affine), os.path.join(session_dir, "extra_data", "magnitude_combined.nii"))

# Reslice T1w images

In [9]:
for i in range(len(t1_files)):
    print(t1_files[i])
    t1_nii = nib.load(t1_files[i])
    mag_nii = nib.load(mag_files[i])
    t1_resampled = nilearn.image.resample_img(t1_nii, target_affine=mag_nii.affine, target_shape=mag_nii.header.get_data_shape())
    filename = t1_files[i].replace(".nii", "_resampled.nii")
    nib.save(t1_resampled, filename)

bids-new2/sub-z0182923/ses-20230705/extra_data/sub-z0182923_ses-20230705_run-01_T1w.nii


/home/uqaste15/miniconda3/envs/prostate/lib/python3.8/site-packages/nilearn/image/resampling.py:597: UserWarning: Casting data from int16 to float32
  warnings.warn(f"Casting data from {data.dtype.name} to {aux}")


# T2* Maps

In [72]:
!cat {session_dirs[0]}/anat/*phase*json | grep -i echotime\"

	"EchoTime": 0.00246,
	"EchoTime": 0.00492,
	"EchoTime": 0.00738,
	"EchoTime": 0.00984,
	"EchoTime": 0.0123,
	"EchoTime": 0.0148,
	"EchoTime": 0.01722,


In [6]:
for session_dir in session_dirs:
    print(session_dir)
    !./make_t2starmaps.jl --magnitude {os.path.join(session_dir, "anat", "*mag*.nii*")} --TEs "[0.00246, 0,00492, 0.00738, 0,00984, 0.0123, 0.0148, 0.01722]" --t2starmap {os.path.join(session_dir, "extra_data", "t2starmap.nii")} --r2starmap {os.path.join(session_dir, "extra_data", "r2starmap.nii")}

bids-new/sub-z0034542/ses-20220715
bids-new/sub-z0182923/ses-20230705
bids-new/sub-z0186251/ses-20221107
bids-new/sub-z0237546/ses-20230508
bids-new/sub-z0317485/ses-20230517
bids-new/sub-z0445614/ses-20230510
bids-new/sub-z0705200/ses-20230104
bids-new/sub-z0755228/ses-20211108
bids-new/sub-z1167038/ses-20220315
bids-new/sub-z1181657/ses-20220315
bids-new/sub-z1262112/ses-20220314
bids-new/sub-z1472355/ses-20221222
bids-new/sub-z1568577/ses-20230510
bids-new/sub-z1571613/ses-20230519
bids-new/sub-z1728751/ses-20220328
bids-new/sub-z1778013/ses-20220715
bids-new/sub-z1818796/ses-20230313
bids-new/sub-z2007565/ses-20220715
bids-new/sub-z2900812/ses-20230523
bids-new/sub-z2904752/ses-20220826
bids-new/sub-z2952003/ses-20230614
bids-new/sub-z3171177/ses-20230313
bids-new/sub-z3278008/ses-20211109
bids-new/sub-z3393287/ses-20230524


# SWI

In [ ]:
!julia -e "import Pkg; Pkg.add(Pkg.PackageSpec(url=\"https://github.com/korbinian90/CLEARSWI.jl\"))"

In [ ]:
#./make_swis.jl --phase bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-phase_MEGRE.nii --magnitude bids-new/sub-z0034542/ses-20220715/anat/sub-z0034542_ses-20220715_run-01_echo-*_part-mag_MEGRE.nii --TEs [0.00492,0.00738] --swi-out swi-out.nii --mip-out mip-out.nii

In [12]:
for session_dir in session_dirs:
    !echo {session_dir}
    !./make_swis.jl \
        --magnitude {os.path.join(session_dir, "anat", "*mag*.nii*")} \
        --phase {os.path.join(session_dir, "anat", "*phase*.nii*")} \
        --TEs "[0.00246, 0.00492, 0.00738, 0.00984, 0.0123, 0.0148, 0.01722]" \
        --swi-out {os.path.join(session_dir, "extra_data", "swi.nii")}

bids-new2/sub-z0182923/ses-20230705


Any["bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-01_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-02_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-03_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-04_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-05_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-06_part-mag_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-07_part-mag_MEGRE.nii"]Any["bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-01_part-phase_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z0182923_ses-20230705_run-01_echo-02_part-phase_MEGRE.nii", "bids-new2/sub-z0182923/ses-20230705/anat/sub-z018292